In [1]:
import numpy as np
import cv2 as cv
import scipy.io
import sys
from matplotlib import pyplot as plt

# Auxiliary functions

In [2]:


# Return the skew matrix formed from a 3x1 vector
def skew(vector):
    return np.array([[0, -vector[2], vector[1]],
                     [vector[2], 0, -vector[0]],
                     [-vector[1], vector[0], 0]])


# Estimate Essential Matrix
def estimate_essential (ptsl, ptsr):

    npoints = ptsl.shape[1] #number of columns

    if ptsr.shape[1] != npoints:
        raise ValueError("Number of points don't match.")

    # Essential Metrix Estimation
    # A_nx9.Es_9x1 = 0

    A = np.zeros((npoints, 9))

    if npoints < 9:
        print('Too few mesurements')
        sys.exit(0)

    # Stack the equation for each pair of matchings
    for i in range (npoints):

        # A[i] = [ptsl[0,i]*ptsr[0,i], ptsl[0,i]*ptsr[1,i], ptsl[0,i]*ptsr[2,i],
        #         ptsl[1,i]*ptsr[0,i], ptsl[1,i]*ptsr[1,i], ptsl[1,i]*ptsr[2,i],
        #         ptsl[2,i]*ptsr[0,i], ptsl[2,i]*ptsr[1,i], ptsl[2,i]*ptsr[2,i] ]

        A[i,:] = np.kron(ptsl[:,i],ptsr[:,i])

    # Check the matrix rank
    r = np.linalg.matrix_rank(A)

    if r < 8:
        print('Measurement matrix rank defficient')
        sys.exit(0)


    # Compute the SVD of matrix A
    U,S,V = np.linalg.svd(A, full_matrices=True)
    # Pick the eigenvector corresponding to the smallest eigenvalue
    # Define the first estimation for the Essential matrix
    E1 = V[-1].reshape(3,3).T

    # Compute the SVD of the Essential Matrix
    U,S,V = np.linalg.svd(E1,full_matrices=True)
    # Substitute the diagonal matrix to project the first estimation into the
    # Essential Matrix Space/Domain
    S = np.diag([1,1,0])
    # Compute the new Essential Matrix belonging to the right domain
    E = U.dot(S).dot(V)
    # or
    #E = np.dot(U,np.dot(S,V))

    return E

def estimate_normalized_essential (ptsl, ptsr):

    ptsl_n, Tl = normalize(ptsl)
    ptsr_n, Tr = normalize(ptsr)
    # Estimate essential matrix
    E = estimate_essential (ptsl_n, ptsr_n)
    # reverse normalization
    E = np.dot(Tr.T,np.dot(E,Tl))

    return E/E[2,2]



def calculate_4solutions(E):
# Calculate the four possible solutions
    #print("E: \n", E)

    # Rotation matrix around the Z-axis of pi/2 radians
    W = np.array([[0,-1,0],[1,0,0],[0,0,1]])

    # SVD of the Essential Matrix
    U,S,Vt = np.linalg.svd(E)

    if np.linalg.det(np.dot(U,Vt))<0:
        Vt = -Vt
    # Compute the four possible solutions
    # Two options for rotation:
    #    * R = U·W·Vt
    #    * R = U·W^T·Vt

    # Two option for translation: T and -T
    # T = U[:,2] and -T = -U[:,2]

    RT = [np.vstack((np.dot(U,np.dot(W,Vt)).T,U[:,2])).T,
          np.vstack((np.dot(U,np.dot(W,Vt)).T,-U[:,2])).T,
          np.vstack((np.dot(U,np.dot(W.T,Vt)).T,U[:,2])).T,
          np.vstack((np.dot(U,np.dot(W.T,Vt)).T,-U[:,2])).T]

    print('Print the four solutions:\n')
    print(RT[0] ,'\n')
    print(RT[1] ,'\n')
    print(RT[2] ,'\n')
    print(RT[3] ,'\n')

    return RT


# Normalize the points
def normalize(pts):

    # to be sure that the last line is all of ones - homogeneous coordinates
    pts = pts / pts[2]

    npoints = pts.shape[1] # number of columns
    # Calculate the Centroid
    #centx = np.sum (pts[0,:])/npoints
    #centy = np.sum (pts[1,:])/npoints
    centroid = np.mean(pts[:2],axis=1)
    # Calculation of the  mean distance in relation to the centroid
    dist_med = sum(np.sqrt((pts[0,:] - centroid[0])**2 + (pts[1,:] - centroid[1])**2))/npoints
    # Scale to make the mean distance equal to  sqrt(2)
    esc = np.sqrt(2)/dist_med
    esc = np.sqrt(2)/np.std(pts[:2])
    # Normalization matrix
    T = np.array([[esc, 0, -esc*centroid[0]],[0, esc, -esc*centroid[1]],[ 0, 0, 1]])

    #Normalized points
    pts_norm = np.dot(T,pts)

    return pts_norm,T


# Drawlines in the images
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,ch = img1.shape

    img1 = cv.cvtColor(img1,cv.COLOR_RGB2GRAY)
    img2 = cv.cvtColor(img2,cv.COLOR_RGB2GRAY)
    img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
    img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)

    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv.circle(img1,tuple([int(pt1[0]),int(pt1[1])]),5,color,-1)
        img2 = cv.circle(img2,tuple([int(pt2[0]),int(pt2[1])]),5,color,-1)
    return img1,img2




def draw_epipolarlines (img1,img2,pts1,pts2,F):

    r,c,ch = img1.shape

    img1 = cv.cvtColor(img1,cv.COLOR_RGB2GRAY)
    img2 = cv.cvtColor(img2,cv.COLOR_RGB2GRAY)
    img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
    img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)

    # Find epilines corresponding to points in right image (second image) and
    # drawing its lines on left image
    lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)
    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)
    fig=plt.figure(figsize=(18,10))
    fig.add_subplot(1,2,1)
    plt.imshow(img5)
    fig.add_subplot(1,2,2)
    plt.imshow(img3)

    return



def triangulate_point(ptsl,ptsr,Pl,Pr):
    """ Point pair triangulation from
    least squares solution. """
    M = np.zeros((6,6))
    M[:3,:4] = Pl
    M[3:,:4] = Pr
    M[:3,4] = -ptsl
    M[3:,5] = -ptsr
    U,S,V = np.linalg.svd(M)
    X = V[-1,:4]
    return X / X[3]

def triangulate(ptsl,ptsr,Pl,Pr):
    """ Two-view triangulation of points in
    x1,x2 (3*n homog. coordinates). """

    n = ptsl.shape[1]
    if ptsr.shape[1] != n:
      raise ValueError("Number of points don't match.")

    X = [ triangulate_point(ptsl[:,i],ptsr[:,i],Pl,Pr) for i in range(n)]

    return np.array(X).T

def select_solution (ptsl,ptsr,Pl,Pr):

    # pick the solution with points in front of cameras
    ind = 0
    maxres = 0
    for i in range(4):
      # triangulate inliers and compute depth for each camera
      X = triangulate(ptsl,ptsr,Pl,Pr[i])
      d1 = np.dot(Pl,X)[2]
      d2 = np.dot(Pr[i],X)[2]

      if (np.sum(d1>0)+np.sum(d2>0)) > maxres:
          maxres = np.sum(d1>0)+np.sum(d2>0)
          ind = i
          infront = (d1>0) & (d2>0)

    Tfinal = Pr[ind][:,3]
    Rfinal = Pr[ind][:,0:3]

    return ind, Tfinal, Rfinal


# Check the positive depth constraint
def verify_positive_depth(ptsmetric_left,ptsmetric_right,RT):
    ##########
    # Select the correct solution
    ####
    # pick the correct solution based on positive depth constraint
    # there are two ways (below 2. is used):
    # 1. Compute both scales and pick the solution where the majority is
    #    positive in both frames
    # 2. Compute volume, which has to be positive if the two scales have
    #    the same sign and then check whether one of the scale is positive
    #    (similar solution suggested by Kanatani, 1993 book).
    ######################
    lin,col = ptsmetric_left.shape
    npoints = col
    index = 0
    posdepth = np.zeros(4)

    for i in range(4):
        alpha1 = np.zeros((npoints,1))
        alpha2 = np.zeros((npoints,1))

        R = RT[i][:,0:3]
        t = RT[i][:,3]

        for j in range(npoints):
            ptsR_hat = skew(ptsmetric_right[:,j])
            ptsL = ptsmetric_left[:,j]
            ptsR = ptsmetric_right[:,j]
            RptsL_hat = skew(R.dot(ptsL))

            alpha1[j] = ((-ptsR_hat.dot(t)).T).dot(ptsR_hat.dot(R).dot(ptsL))/(np.linalg.norm(ptsR_hat.dot(t))**2)
            alpha2[j] = (RptsL_hat.dot(ptsR)).T.dot(RptsL_hat.dot(t))/(np.linalg.norm(RptsL_hat.dot(ptsR))**2)

        depth = np.sum(alpha1>0)
        depth2 = np.sum(alpha2>0)
        posdepth[i] = depth + depth2


    index = np.argmax(posdepth)

    Tfinal = RT[index][:,3]
    Rfinal = RT[index][:,0:3]
    return index,Tfinal,Rfinal




# Example Cases

In [ ]:
def case_one():
    # Read images
    imgl = cv.imread('img_left.jpg')  #queryimage # left image
    imgr = cv.imread('img_right.jpg') #trainimage # right image

    # intrinsic parameter matrix

    # for the right camera
    Kr = np.array([[ 857.1590, 0, 318.2537],[0, 860.8839, 233.3245],[0, 0, 1.0000]])

    # for the left camera
    Kl = np.array([[855.3492, 0, 319.2672],[0, 859.2368, 247.6019],[0, 0, 1.0000]])


    # Extrinsic Parameters
    # Translation
    Tr = np.array([-118.1274, 0.5238, 0.1311])
    Tr = Tr/np.linalg.norm(Tr)
    T_hat = skew(Tr)
    # Rotation
    Rmat = np.array([[ 0.9999,   0.0009,    0.0147],[ -0.0010,    0.9999,    0.0110],[-0.0147,   -0.0110,    0.9998]])


    # Read points that were collected manually
    data = scipy.io.loadmat('selected_points.mat')

    ptsl = data['npt0']
    ptsr = data['npt1']

    return imgl,imgr,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat

def case_two():
    #Read images
    imgl = cv.imread('colorLeft.png')  #queryimage # left image
    imgr = cv.imread('colorRight.png') #trainimage # right image

    # intrinsic parameter matrix

    fm = 403.657593; # Fical distantce in pixels
    cx = 161.644318; # Principal point - x-coordinate (pixels)
    cy = 124.202080; # Principal point - y-coordinate (pixels)
    bl = 119.929; # baseline (mm)
    # for the right camera
    Kr = np.array([[ fm, 0, cx],[0, fm, cy],[0, 0, 1.0000]])

    # for the left camera
    Kl = np.array([[fm, 0, cx],[0, fm, cy],[0, 0, 1.0000]])

    # Extrinsec parameters
    # Translation between cameras
    Tr = np.array([-bl, 0, 0])
    Tr = Tr/np.linalg.norm(Tr)
    T_hat = skew(Tr)
    # Rotation
    Rmat = np.array([[ 1,0,0],[ 0,1,0],[0,0,1]])



    # find the keypoints and descriptors with SIFT
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(imgl,None)
    kp2, des2 = sift.detectAndCompute(imgr,None)

    # Solve matching between detected points in both images
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    good = []
    ptsl = []
    ptsr = []

    # Select good matches
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):

        if m.distance < 0.7*n.distance:
            good.append(m)
            ptsr.append(kp2[m.trainIdx].pt)
            ptsl.append(kp1[m.queryIdx].pt)


    ptsl = np.int32(ptsl).T
    ptsr = np.int32(ptsr).T

    return imgl,imgr,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat

def case_three():

    #Read images
    imgl = cv.imread('270_left.jpg')  #queryimage # left image
    imgr = cv.imread('270_right.jpg') #trainimage # right image

    img_size = imgl.shape
    imgl = cv.resize(imgl, (int(img_size[1]/4), int(img_size[0]/4)))

    img_size = imgr.shape
    imgr = cv.resize(imgr, (int(img_size[1]/4), int(img_size[0]/4)))


    Kl = np.array([[ 2.24534083e+03, -1.86235000e-01,  1.00832942e+03],
                   [ 0.00000000e+00,  2.24106139e+03,  1.02170932e+03],
                   [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

    Kr = np.array([[2.24011968e+03, 2.23877000e-01, 1.01598269e+03],
                   [0.00000000e+00, 2.23629070e+03, 1.03758278e+03],
                   [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

    # Correct intrinsic parameters due to resizing images
    Kl = Kl/4
    Kl [-1,-1] = 1
    Kr = Kr/4
    Kr [-1,-1] = 1

    Rmat = np.array([[ 0.999995, -0.002636, -0.001449],
                     [ 0.002633,  0.999995, -0.001937],
                     [ 0.001455,  0.001933,  0.999997]])
    Tr = np.array([-4.17709527e+02,-8.92217000e-01,-1.37416000e-01])
    Tr = Tr/np.linalg.norm(Tr)
    T_hat = skew(Tr)

    # find the keypoints and descriptors with SIFT
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(imgl,None)
    kp2, des2 = sift.detectAndCompute(imgr,None)

    # Solve matching between detected points in both images
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    good = []
    ptsl = []
    ptsr = []

    # Select good matches
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):

        if m.distance < 0.7*n.distance:
            good.append(m)
            ptsr.append(kp2[m.trainIdx].pt)
            ptsl.append(kp1[m.queryIdx].pt)


    ptsl = np.int32(ptsl).T
    ptsr = np.int32(ptsr).T

    return imgl,imgr,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat






# Main program with the 3 examples

In [ ]:
##################### Main ###################


CASE = 3

if CASE == 1:
    imgl0,imgr0,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat = case_one()

if CASE == 2:
    imgl0,imgr0,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat = case_two()

if CASE == 3:
    imgl0,imgr0,Kl,Kr,ptsl,ptsr,T_hat,Tr,Rmat = case_three()





# Show images and plot points
imgl = cv.cvtColor(imgl0,cv.COLOR_BGR2RGB)
imgr = cv.cvtColor(imgr0, cv.COLOR_BGR2RGB)
ydim = imgl.shape[0]
xdim = imgl.shape[1]


plt.figure(figsize=(18,10))
plt.subplot(121)
plt.imshow(imgl)
plt.plot(ptsl[0,:],ptsl[1,:],'b*')
plt.subplot(122)
plt.imshow(imgr)
plt.plot(ptsr[0,:],ptsr[1,:],'m*')


# compute image coordinates in metric
Kl_inv = np.linalg.inv(Kl)
Kr_inv = np.linalg.inv(Kr)

lin,col = ptsl.shape
npoints = col

ptsl_m = np.dot(Kl_inv,np.vstack((ptsl,np.ones((1,npoints)))))
ptsr_m = np.dot(Kr_inv,np.vstack((ptsr,np.ones((1,npoints)))))


#Estimate the Essential matrix using our method
print('\n##############################################')
print('Essential Matrix estimation using our method')
print('##############################################\n')

E = estimate_normalized_essential(ptsl_m, ptsr_m)

print('Essential Matrix')
print(E,'\n')

# Calculate the four possible solutions
RT = calculate_4solutions(E)

# Select the right solution using the positive depth constraint
index,Tfinal,Rfinal = verify_positive_depth(ptsl_m,ptsr_m,RT)

P1 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
ind_test,T_test,R_test = select_solution (ptsl_m,ptsr_m,P1,RT)

print('Final solution using our method has the index: ', index)
print(Tfinal)
print(Rfinal,'\n')

print('Another solution using triangulation: ', ind_test)
print(T_test)
print(R_test)

# Draw epipolar lines
pts1 = ptsl[0:2,:].T
pts2 = ptsr[0:2,:].T
# Compute the Fundamental Matrix
F = (np.linalg.inv(Kr.T)).dot(E).dot(Kl_inv)   # F = (Kr^-T).E.(Kl^-1)

draw_epipolarlines (imgl,imgr,pts1,pts2,F)


print('\n##############################################')
print('Essential Matrix estimation using OpenCV')
print('##############################################\n')

# Estimate the Fundamental matrix
# cv.FM_8POINT + cv.FM_RANSAC
pts1 = ptsl[0:2,:].T
pts2 = ptsr[0:2,:].T

# Estimate the Fundamental matrix
# cv.FM_8POINT + cv.FM_RANSAC
F, mask = cv.findFundamentalMat(pts1,pts2,cv.RANSAC,1,0.9999999)
# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

# Compute the Essential Matrix
EopenCV = (Kr.T).dot(F).dot(Kl)
EopenCV = EopenCV /EopenCV [2,2]
print('Essential Matrix - OpenCV')
print(EopenCV,'\n')

# Calculate the four possible solutions
RT = calculate_4solutions(EopenCV)

# Select the right solution using the positive depth constraint
index,Tfinal,Rfinal = verify_positive_depth(ptsl_m,ptsr_m,RT)
P1 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
ind_test,T_test,R_test = select_solution (ptsl_m,ptsr_m,P1,RT)

print('Final solution using our method has the index: ', index)
print(Tfinal)
print(Rfinal,'\n')

print('Another solution using triangulation: ', ind_test)
print(T_test)
print(R_test)

# Draw epipolar lines
pts1 = ptsl[0:2,:].T
pts2 = ptsr[0:2,:].T
# Compute the Fundamental Matrix
F = (np.linalg.inv(Kr.T)).dot(EopenCV).dot(Kl_inv)

draw_epipolarlines (imgl,imgr,pts1,pts2,F)


# Compute the real Essential Matrix
Ereal = T_hat.dot(Rmat)
print('\n##############################################')
print('Real Solution')
print('##############################################\n')
print('E:\n',Ereal,'\n')
print('R:\n',Rmat,'\nT:\n',Tr)

pts1 = ptsl[0:2,:].T
pts2 = ptsr[0:2,:].T

# Compute the Fundamental Matrix
F = (np.linalg.inv(Kr.T)).dot(Ereal).dot(Kl_inv)

draw_epipolarlines (imgl,imgr,pts1,pts2,F)


plt.show()



Output hidden; open in https://colab.research.google.com to view.